# BBox2YOLO

Changle labelleing format from default BBox to YOLO

In [1]:
from shutil import copyfile
from PIL import Image
import os
import random
import pandas as pd
import cv2

In [2]:
images_dir = "yolo_signs_images/"
labels_dir = "yolo_signs_labels/"
output_dir = "yolo_signs/"

classes = [
    "Dont_Enter",
    "Turn_Left",
    "Keep_Left",
    "Straight_Right",
    "Crosswalk",
    "Loose_Chippings",
    "Roadworks",
    "Park",
    "Pedestrian",
    "Lights_Green",
    "Lights_Red",
    "Park_Area",
    "Handicap"
]

In [3]:
# write class names
with open(output_dir + "class.names", "w") as f:
    for name in classes:
        f.write(name + "\n")

# write data file        
with open(output_dir + "yolo_signs.data", "w") as f:
    f.write("classes = " + str(len(classes)) + "\n")
    f.write("train = /home/ubuntu/workspace/darknet/data/yolo_signs/train.txt" + "\n")
    f.write("valid = /home/ubuntu/workspace/darknet/data/yolo_signs/test.txt" + "\n")
    f.write("names = /home/ubuntu/workspace/darknet/data/yolo_signs/class.names" + "\n")
    f.write("backup = /home/ubuntu/workspace/darknet/backup/" + "\n")

In [4]:
df = pd.DataFrame(columns=['Path', 'ClassId', 'Roi.X1', 'Roi.Y1', 'Roi.X2', 'Roi.Y2'])

for root, dirs, files in os.walk(labels_dir):
        for file in files:
            label_path = os.path.join(root,file)
            image_path = label_path.replace(labels_dir, images_dir)
            
            #print(label_path)
            
            lbl_str = label_path.split("/")[1]
            lbl_id = classes.index(lbl_str)
            
            nmbr = os.path.basename(image_path).split('.')[0]
            with open(label_path, "r") as f:
                lines = f.readlines()
                if len(lines) > 1:
                    for l in range(1, len(lines)):
                        tmp = lines[l].split(" ")
                        df = df.append({'Path': nmbr, 'ClassId': lbl_id, 'Roi.X1': int(tmp[0]), 'Roi.Y1': int(tmp[1]), 'Roi.X2': int(tmp[2]), 'Roi.Y2': int(tmp[3])}, ignore_index=True)
            
print(len(df))
print(df[0:5])

10793
     Path ClassId Roi.X1 Roi.Y1 Roi.X2 Roi.Y2
0  090938       9    699    245    733    309
1  090336       9    644    262    673    316
2  090302       9    718    220    778    318
3  090609       9    658    253    687    308
4  090277       9    784    234    831    304


In [5]:
# preprocess training dataset
w = float(1280)
h = float(720)
    
for i, index in enumerate(df.index):
    c = df.ClassId[index]
    n = df.Path[index]
    x1 = df["Roi.X1"][index]
    y1 = df["Roi.Y1"][index]
    x2 = df["Roi.X2"][index]
    y2 = df["Roi.Y2"][index]
    
    ox = ((x1 + x2)/2)/w
    oy = ((y1 + y2)/2)/h
    ow = abs(x1 - x2)/w
    oh = abs(y1 - y2)/h
    
    string = str(c) + " " + str(ox) + " " + str(oy) + " " + str(ow) + " " + str(oh)
    
    print(n, string)
    with open(output_dir + n + ".txt", "a") as f:
        f.write(string + "\n")

('090938', '9 0.559375 0.384722222222 0.0265625 0.0888888888889')
('090336', '9 0.5140625 0.401388888889 0.02265625 0.075')
('090302', '9 0.584375 0.373611111111 0.046875 0.136111111111')
('090609', '9 0.525 0.388888888889 0.02265625 0.0763888888889')
('090277', '9 0.63046875 0.373611111111 0.03671875 0.0972222222222')
('090495', '9 0.54921875 0.386111111111 0.03203125 0.0861111111111')
('090806', '9 0.5921875 0.3625 0.0453125 0.130555555556')
('090273', '9 0.52421875 0.390277777778 0.02578125 0.0833333333333')
('090766', '9 0.5015625 0.391666666667 0.02109375 0.0680555555556')
('090301', '9 0.53359375 0.393055555556 0.0203125 0.0680555555556')
('090821', '9 0.55390625 0.386111111111 0.02890625 0.0888888888889')
('090698', '9 0.55703125 0.376388888889 0.03046875 0.0875')
('090723', '9 0.49453125 0.397222222222 0.02265625 0.0652777777778')
('090187', '9 0.5578125 0.386111111111 0.03125 0.0972222222222')
('091178', '9 0.50546875 0.398611111111 0.01953125 0.0666666666667')
('090570', '9 0

('000633', '0 0.5828125 0.4125 0.02890625 0.0583333333333')
('000355', '0 0.55078125 0.409722222222 0.01796875 0.0277777777778')
('000352', '0 0.65625 0.391666666667 0.0421875 0.0736111111111')
('000718', '0 0.571875 0.409722222222 0.028125 0.0402777777778')
('001175', '0 0.5359375 0.420833333333 0.02421875 0.0388888888889')
('000613', '0 0.5828125 0.3875 0.03671875 0.0638888888889')
('000236', '0 0.58984375 0.391666666667 0.01953125 0.0305555555556')
('001290', '0 0.534375 0.422222222222 0.02890625 0.0444444444444')
('000673', '0 0.6984375 0.3625 0.0953125 0.166666666667')
('000342', '0 0.60546875 0.373611111111 0.04453125 0.0819444444444')
('000470', '0 0.51796875 0.425 0.01796875 0.0305555555556')
('000474', '0 0.5578125 0.394444444444 0.02890625 0.0527777777778')
('000592', '0 0.5625 0.405555555556 0.03515625 0.0611111111111')
('000687', '0 0.49609375 0.431944444444 0.02109375 0.0263888888889')
('000324', '0 0.73515625 0.380555555556 0.0578125 0.0972222222222')
('000957', '0 0.5382

('110474', '11 0.52578125 0.573611111111 0.07265625 0.0347222222222')
('110474', '11 0.796875 0.611111111111 0.096875 0.0416666666667')
('110857', '11 0.5484375 0.595833333333 0.08671875 0.0513888888889')
('110857', '11 0.32109375 0.573611111111 0.07890625 0.0402777777778')
('110394', '11 0.36875 0.601388888889 0.05703125 0.0305555555556')
('110394', '11 0.64375 0.631944444444 0.0546875 0.0430555555556')
('110011', '11 0.58515625 0.8625 0.2 0.194444444444')
('110011', '11 0.1140625 0.740277777778 0.18359375 0.0972222222222')
('110627', '11 0.7140625 0.697222222222 0.0765625 0.0569444444444')
('110627', '11 0.37421875 0.613888888889 0.06171875 0.0347222222222')
('110389', '11 0.334375 0.595833333333 0.0921875 0.0402777777778')
('110389', '11 0.54453125 0.556944444444 0.071875 0.0291666666667')
('110320', '11 0.22890625 0.675 0.15859375 0.0694444444444')
('110033', '11 0.84453125 0.740277777778 0.1828125 0.1125')
('110221', '11 0.609375 0.694444444444 0.1484375 0.0875')
('110304', '11 0.

('110717', '11 0.69375 0.630555555556 0.10078125 0.0513888888889')
('110607', '11 0.5546875 0.6 0.09375 0.0402777777778')
('110607', '11 0.921875 0.677777777778 0.13984375 0.0833333333333')
('110333', '11 0.64921875 0.630555555556 0.121875 0.0625')
('110560', '11 0.565625 0.691666666667 0.121875 0.0736111111111')
('110560', '11 0.19765625 0.658333333333 0.12109375 0.0625')
('110289', '11 0.2453125 0.854166666667 0.1 0.109722222222')
('110289', '11 0.690625 0.683333333333 0.07890625 0.0527777777778')
('110454', '11 0.49140625 0.901388888889 0.23125 0.173611111111')
('110581', '11 0.425 0.55 0.0609375 0.0236111111111')
('110581', '11 0.628125 0.570833333333 0.0734375 0.0319444444444')
('110390', '11 0.80390625 0.565277777778 0.03984375 0.0319444444444')
('110367', '11 0.12734375 0.651388888889 0.06328125 0.0388888888889')
('110087', '11 0.5734375 0.620833333333 0.1015625 0.0625')
('110087', '11 0.94609375 0.705555555556 0.10546875 0.0861111111111')
('110646', '11 0.62421875 0.65694444444

('120225', '12 0.63984375 0.905555555556 0.2515625 0.186111111111')
('120232', '12 0.49453125 0.755555555556 0.1609375 0.104166666667')
('120733', '12 0.36171875 0.616666666667 0.08671875 0.0416666666667')
('120735', '12 0.29140625 0.602777777778 0.08671875 0.0333333333333')
('120887', '12 0.4875 0.938888888889 0.17734375 0.122222222222')
('120847', '12 0.6203125 0.6125 0.07578125 0.0361111111111')
('120680', '12 0.7453125 0.726388888889 0.16796875 0.120833333333')
('120354', '12 0.5453125 0.626388888889 0.09921875 0.0541666666667')
('120949', '12 0.4828125 0.658333333333 0.10703125 0.0652777777778')
('120318', '12 0.41640625 0.6625 0.1078125 0.0666666666667')
('121068', '12 0.48671875 0.827777777778 0.19453125 0.155555555556')
('120820', '12 0.4296875 0.622222222222 0.07421875 0.0333333333333')
('120438', '12 0.4375 0.665277777778 0.115625 0.0680555555556')
('120834', '12 0.8671875 0.726388888889 0.18515625 0.0930555555556')
('120781', '12 0.60703125 0.673611111111 0.1109375 0.0694444

...

('100706', '10 0.646875 0.351388888889 0.065625 0.180555555556')
('101083', '10 0.61328125 0.372222222222 0.04296875 0.115277777778')
('100864', '10 0.5359375 0.409722222222 0.01640625 0.0513888888889')
('101129', '10 0.52265625 0.427777777778 0.01484375 0.0472222222222')
('100557', '10 0.5984375 0.393055555556 0.0171875 0.0527777777778')
('100819', '10 0.61484375 0.376388888889 0.0453125 0.131944444444')
('101101', '10 0.49921875 0.406944444444 0.015625 0.0458333333333')
('100498', '10 0.54609375 0.3875 0.02265625 0.0652777777778')
('101149', '10 0.5703125 0.381944444444 0.03203125 0.0944444444444')
('101109', '10 0.4609375 0.422222222222 0.015625 0.0486111111111')
('100901', '10 0.72578125 0.311111111111 0.10078125 0.283333333333')
('101114', '10 0.621875 0.368055555556 0.0515625 0.15')
('100613', '10 0.575 0.398611111111 0.021875 0.0611111111111')
('100609', '10 0.478125 0.4 0.01953125 0.05')
('101082', '10 0.62265625 0.336111111111 0.06640625 0.191666666667')
('100722', '10 0.48671

('060529', '6 0.58359375 0.422222222222 0.02734375 0.0388888888889')
('060529', '6 0.5046875 0.493055555556 0.0203125 0.0305555555556')
('060723', '6 0.54375 0.488888888889 0.015625 0.0208333333333')
('060723', '6 0.60546875 0.4375 0.02109375 0.0291666666667')
('061171', '6 0.49921875 0.6375 0.0859375 0.118055555556')
('060602', '6 0.5171875 0.555555555556 0.0359375 0.0513888888889')
('060602', '6 0.67109375 0.441666666667 0.06171875 0.0847222222222')
('061080', '6 0.51875 0.545833333333 0.03515625 0.0527777777778')
('061080', '6 0.66796875 0.430555555556 0.0625 0.0833333333333')
('060594', '6 0.490625 0.508333333333 0.03203125 0.0486111111111')
('060594', '6 0.61484375 0.404166666667 0.0515625 0.0736111111111')
('060419', '6 0.7953125 0.798611111111 0.178125 0.220833333333')
('060557', '6 0.62265625 0.430555555556 0.03515625 0.0486111111111')
('060557', '6 0.525 0.513888888889 0.02578125 0.0361111111111')
('061053', '6 0.4546875 0.618055555556 0.0703125 0.1')
('060548', '6 0.42421875 

('020500', '2 0.5265625 0.4125 0.03046875 0.0583333333333')
('021280', '2 0.3703125 0.393055555556 0.0578125 0.111111111111')
('021280', '2 0.4453125 0.365277777778 0.08671875 0.155555555556')
('020819', '2 0.459375 0.408333333333 0.0296875 0.0569444444444')
('020819', '2 0.5015625 0.404166666667 0.03828125 0.0652777777778')
('020275', '2 0.5109375 0.401388888889 0.0484375 0.0833333333333')
('020275', '2 0.60078125 0.377777777778 0.07734375 0.141666666667')
('020900', '2 0.409375 0.427777777778 0.02421875 0.0430555555556')
('020900', '2 0.44296875 0.425 0.03125 0.05')
('020743', '2 0.4203125 0.372222222222 0.065625 0.116666666667')
('020743', '2 0.5234375 0.3375 0.115625 0.205555555556')
('021291', '2 0.47265625 0.4375 0.025 0.0513888888889')
('021291', '2 0.51484375 0.430555555556 0.0328125 0.0638888888889')
('020557', '2 0.46171875 0.441666666667 0.04453125 0.0736111111111')
('020557', '2 0.52109375 0.431944444444 0.05390625 0.104166666667')
('021094', '2 0.490625 0.456944444444 0.02

('020976', '2 0.5578125 0.404166666667 0.034375 0.0680555555556')
('021057', '2 0.4796875 0.4125 0.025 0.0555555555556')
('021057', '2 0.525 0.408333333333 0.03046875 0.0583333333333')
('021108', '2 0.44453125 0.411111111111 0.025 0.0513888888889')
('021108', '2 0.48046875 0.406944444444 0.0265625 0.05')
('020965', '2 0.50234375 0.423611111111 0.0296875 0.0444444444444')
('020965', '2 0.46640625 0.426388888889 0.02265625 0.0416666666667')
('021303', '2 0.51640625 0.481944444444 0.01953125 0.0361111111111')
('021303', '2 0.54140625 0.480555555556 0.0234375 0.0402777777778')
('020994', '2 0.5296875 0.470833333333 0.01953125 0.0375')
('020994', '2 0.55625 0.469444444444 0.02421875 0.0444444444444')
('021033', '2 0.39375 0.420833333333 0.034375 0.0597222222222')
('021033', '2 0.4453125 0.411111111111 0.0453125 0.0777777777778')
('021208', '2 0.4796875 0.425 0.034375 0.0611111111111')
('021208', '2 0.5390625 0.423611111111 0.04609375 0.0902777777778')
('021328', '2 0.4046875 0.418055555556 

('070914', '7 0.915625 0.275 0.16328125 0.398611111111')
('070198', '7 0.59765625 0.427777777778 0.0375 0.0680555555556')
('070596', '7 0.69296875 0.4125 0.05703125 0.0944444444444')
('070619', '7 0.6375 0.483333333333 0.03359375 0.0611111111111')
('070829', '7 0.5046875 0.416666666667 0.04375 0.0888888888889')
('070343', '7 0.58515625 0.423611111111 0.03359375 0.0583333333333')
('070533', '7 0.64765625 0.390277777778 0.06171875 0.113888888889')
('070421', '7 0.88515625 0.3875 0.08203125 0.143055555556')
('071048', '7 0.7234375 0.373611111111 0.11015625 0.208333333333')
('070781', '7 0.590625 0.493055555556 0.03125 0.0583333333333')
('070491', '7 0.81484375 0.476388888889 0.03515625 0.0541666666667')
('070539', '7 0.8765625 0.383333333333 0.1875 0.331944444444')
('070459', '7 0.5828125 0.486111111111 0.03671875 0.0694444444444')
('070897', '7 0.58828125 0.484722222222 0.028125 0.0583333333333')
('070413', '7 0.80625 0.286111111111 0.14921875 0.302777777778')
('071096', '7 0.6046875 0.4

In [6]:
random.seed(123)
file_history = set()
with open(output_dir + "train.txt", "w") as f_train, open(output_dir + "test.txt", "w") as f_test:
    for i, index in enumerate(df.index):
        c = df.ClassId[index]
        n = df.Path[index]
        if n not in file_history:
            file_history.add(n)
            copyfile(images_dir + classes[c] + "/" + n + ".jpg", output_dir + n + ".jpg")
            n = "data/yolo_signs/" + n + ".jpg"
            rnd = random.randint(0,4) #Training 75%, Validation 25%
            if rnd == 0:
                f_test.write(n + "\n")
            else:
                f_train.write(n + "\n")

In [7]:
with open(output_dir + "train.txt", "a") as f_train, open(output_dir + "test.txt", "a") as f_test:
    for root, dirs, files in os.walk(output_dir):
        for file in files:
            if file.endswith("jpg"):
                txt = file.replace("jpg","txt")
                if not os.path.isfile(os.path.join(root,txt)):
                    print(txt)
                    open(os.path.join(root,txt), 'a').close()
                    rnd = random.randint(0,4) #Training 75%, Validation 25%
                    if rnd == 0:
                        f_test.write("data/yolo_signs/" + file + "\n")
                    else:
                        f_train.write("data/yolo_signs/" + file + "\n")

12878.txt
02874.txt
03707.txt
02682.txt
01451.txt
04198.txt
11620.txt
00974.txt
03557.txt
00052.txt
00810.txt
03789.txt
00224.txt
12185.txt
11531.txt
03317.txt
02915.txt
00184.txt
02461.txt
11733.txt
03874.txt
04672.txt
10739.txt
02644.txt
00273.txt
11164.txt
01517.txt
00242.txt
02180.txt
12604.txt
00383.txt
12006.txt
10339.txt
02483.txt
01767.txt
12214.txt
02369.txt
12300.txt
12774.txt
04315.txt
12936.txt
04473.txt
00170.txt
12741.txt
01132.txt
04498.txt
10537.txt
12250.txt
12540.txt
11612.txt
11099.txt
10695.txt
04428.txt
03982.txt
00283.txt
03390.txt
03675.txt
00375.txt
04887.txt
12789.txt
10617.txt
10485.txt
04884.txt
03300.txt
12750.txt
02167.txt
10926.txt
01526.txt
00370.txt
02520.txt
10614.txt
10321.txt
04216.txt
02170.txt
02722.txt
03193.txt
12409.txt
01884.txt
10544.txt
12702.txt
01879.txt
02201.txt
01333.txt
03815.txt
00503.txt
01138.txt
00823.txt
02786.txt
01796.txt
10424.txt
03984.txt
03961.txt
02501.txt
00272.txt
03195.txt
10243.txt
01522.txt
00870.txt
00293.txt
00250.txt


03439.txt
10526.txt
10024.txt
02849.txt
02868.txt
10759.txt
00201.txt
04317.txt
03336.txt
12756.txt
01388.txt
10802.txt
02296.txt
02927.txt
00183.txt
01626.txt
01768.txt
02701.txt
12431.txt
03483.txt
01005.txt
00964.txt
00022.txt
00288.txt
10593.txt
10959.txt
01920.txt
11781.txt
12464.txt
01331.txt
02710.txt
11211.txt
02176.txt
11472.txt
02244.txt
02395.txt
01352.txt
00060.txt
03806.txt
11432.txt
01302.txt
02642.txt
04345.txt
00724.txt
00091.txt
00381.txt
12385.txt
12526.txt
11754.txt
04174.txt
00575.txt
02749.txt
12413.txt
11446.txt
10124.txt
12759.txt
12742.txt
00782.txt
03698.txt
04172.txt
02791.txt
00762.txt
04575.txt
10492.txt
01086.txt
12804.txt
10129.txt
11834.txt
01801.txt
03395.txt
11535.txt
02762.txt
10265.txt
02411.txt
02918.txt
11345.txt
01307.txt
12101.txt
12815.txt
04072.txt
02727.txt
10854.txt
10094.txt
11773.txt
00740.txt
02907.txt
03119.txt
02023.txt
04674.txt
00509.txt
03011.txt
11070.txt
03108.txt
01121.txt
04206.txt
00009.txt
11873.txt
02611.txt
11365.txt
04229.txt


04663.txt
01696.txt
01564.txt
02049.txt
01565.txt
03969.txt
11356.txt
04787.txt
12014.txt
10701.txt
12186.txt
10929.txt
02615.txt
04434.txt
04644.txt
01707.txt
12011.txt
03636.txt
11107.txt
01662.txt
00137.txt
02789.txt
10502.txt
03200.txt
03124.txt
03758.txt
10693.txt
12389.txt
01320.txt
02655.txt
00327.txt
03238.txt
12260.txt
01448.txt
03907.txt
01135.txt
04112.txt
10452.txt
03445.txt
10365.txt
03252.txt
00955.txt
00444.txt
01438.txt
11198.txt
02274.txt
03312.txt
11789.txt
01897.txt
12524.txt
02417.txt
02978.txt
03669.txt
02659.txt
11193.txt
02349.txt
10801.txt
01991.txt
00356.txt
12718.txt
03062.txt
01167.txt
04633.txt
11749.txt
03359.txt
11527.txt
00042.txt
04121.txt
10448.txt
11283.txt
12016.txt
01416.txt
12894.txt
10991.txt
03544.txt
00352.txt
02091.txt
02694.txt
04065.txt
01928.txt
04738.txt
04405.txt
02563.txt
11138.txt
01637.txt
00650.txt
12381.txt
00978.txt
00686.txt
00414.txt
01323.txt
11803.txt
03491.txt
02723.txt
01316.txt
01444.txt
04094.txt
12194.txt
11623.txt
11152.txt


00057.txt
03923.txt
03308.txt
03251.txt
00217.txt
03965.txt
10036.txt
03627.txt
00986.txt
10557.txt
10924.txt
00536.txt
00378.txt
12592.txt
04621.txt
03822.txt
02237.txt
10193.txt
03068.txt
01162.txt
10820.txt
00163.txt
02839.txt
00134.txt
03329.txt
04661.txt
12512.txt
00021.txt
11947.txt
00735.txt
03275.txt
11501.txt
04045.txt
10990.txt
10826.txt
01324.txt
02016.txt
12541.txt
11212.txt
10520.txt
00285.txt
10595.txt
10866.txt
02122.txt
01174.txt
01157.txt
02512.txt
00958.txt
11918.txt
04771.txt
03185.txt
02153.txt
12813.txt
03932.txt
03944.txt
12415.txt
04745.txt
00633.txt
00276.txt
04362.txt
10210.txt
12528.txt
00703.txt
04449.txt
12860.txt
10884.txt
11298.txt
04634.txt
02481.txt
11821.txt
03880.txt
04760.txt
10629.txt
01340.txt
01002.txt
01683.txt
11324.txt
02221.txt
04761.txt
12397.txt
02730.txt
03025.txt
04178.txt
03679.txt
03157.txt
03506.txt
00982.txt
12686.txt
01729.txt
10903.txt
11619.txt
03499.txt
04723.txt
03077.txt
11220.txt
02571.txt
03922.txt
02479.txt
00710.txt
10372.txt


04725.txt
03953.txt
11722.txt
11649.txt
03203.txt
12665.txt
10322.txt
04902.txt
12720.txt
11921.txt
02942.txt
01084.txt
02252.txt
00363.txt
03120.txt
04167.txt
02403.txt
00822.txt
03711.txt
00331.txt
00985.txt
11699.txt
12055.txt
12651.txt
02224.txt
10572.txt
00246.txt
02653.txt
03781.txt
04802.txt
03375.txt
03387.txt
00548.txt
11418.txt
02763.txt
01580.txt
12483.txt
12179.txt
12181.txt
02478.txt
04840.txt
00990.txt
02215.txt
12560.txt
00267.txt
02467.txt
03653.txt
03474.txt
11173.txt
12896.txt
00711.txt
04858.txt
00303.txt
03278.txt
04010.txt
12855.txt
03222.txt
10336.txt
10263.txt
00062.txt
12646.txt
11055.txt
00543.txt
00329.txt
03978.txt
12215.txt
02241.txt
02698.txt
02286.txt
00334.txt
12223.txt
04707.txt
02372.txt
00133.txt
00118.txt
04600.txt
01560.txt
04358.txt
00482.txt
01739.txt
12507.txt
01081.txt
11247.txt
10711.txt
04269.txt
01386.txt
10494.txt
00222.txt
01152.txt
02909.txt
02264.txt
04101.txt
12428.txt
12870.txt
11571.txt
00677.txt
03202.txt
00921.txt
03901.txt
01827.txt


12364.txt
02729.txt
11316.txt
10006.txt
04004.txt
03085.txt
00904.txt
04183.txt
03966.txt
00667.txt
02438.txt
10513.txt
03366.txt
04093.txt
10482.txt
01376.txt
04115.txt
11106.txt
00599.txt
03838.txt
10104.txt
01449.txt
01454.txt
12070.txt
12617.txt
02371.txt
00244.txt
12367.txt
02879.txt
02126.txt
10548.txt
00595.txt
03682.txt
10433.txt
11742.txt
11637.txt
04354.txt
00309.txt
04106.txt
03128.txt
03639.txt
00807.txt
12641.txt
03651.txt
12245.txt
10748.txt
01144.txt
11351.txt
11969.txt
12914.txt
00549.txt
03535.txt
10709.txt
04177.txt
10847.txt
04153.txt
01614.txt
00886.txt
10188.txt
04127.txt
00046.txt
03650.txt
02795.txt
02860.txt
10528.txt
02968.txt
01694.txt
12092.txt
10603.txt
12709.txt
00088.txt
10841.txt
04393.txt
03642.txt
04636.txt
03460.txt
10649.txt
00806.txt
11278.txt
00053.txt
03087.txt
04272.txt
00751.txt
04133.txt
11001.txt
01102.txt
03273.txt
11458.txt
11441.txt
02822.txt
11633.txt
03353.txt
00535.txt
11406.txt
02345.txt
10594.txt
11916.txt
04831.txt
11438.txt
00300.txt
